#导入相关路径与包

In [ ]:
pip install timm

In [ ]:
# pip install pytorch-lightning==1.5.6

In [ ]:
import pathlib
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from random import sample
import PIL.Image
import albumentations.pytorch
import cv2
import matplotlib.pyplot as plt

from typing import List, Tuple



BATCH_SIZE = 200
MAX_IMAGE_PER_CLASS = 4
def gpu_unravel(batch):
        input_dict, target_dict = batch
        input_dict = {k: input_dict[k].cuda() for k in input_dict}
        target_dict = {k: target_dict[k].cuda() for k in target_dict}
        return input_dict, target_dict
dict_unravel = gpu_unravel

import sys
import importlib
from types import SimpleNamespace
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.special import softmax
from joblib import Parallel, delayed
import seaborn as sns
from random import sample

name = "config1"
# MODEL_PATH = "E:/download/.kaggle/results/"+f"{name}_ckpt_4.pth"
# configPath = '../configs'
MODEL_PATH = "/kaggle/input/config1/"+f"{name}_ckpt_4.pth"
configPath = '/kaggle/input/config1'
sys.path.append(configPath)
# args = importlib.import_module('config1_test').args #导入对应config中的变量
args = importlib.import_module('config1-test').args #导入对应config中的变量

args =  SimpleNamespace(**args)

args.img_path_train = args.data_path + 'train/'
args.img_path_val = args.data_path_valid + 'valid/'
args.img_path_test = args.data_path + 'test/'



# 削减训练集数量

In [ ]:
train_df = pd.read_csv(args.data_path + args.train_csv_fn)
if len(train_df) == 1580470:# submission use all the training images
    records = {}

    for image_id, landmark_id in train_df.values:
        if landmark_id in records:
            records[landmark_id].append(image_id)
        else:
            records[landmark_id] = [image_id]
        
    image_ids = []
    landmark_ids = []

    for landmark_id, img_ids in records.items():
        num = min(len(img_ids), MAX_IMAGE_PER_CLASS)# maxium two images
        # image_ids.extend(records[landmark_id][:num])
        # landmark_ids.extend([landmark_id] * num)
        image_ids.extend(sample(records[landmark_id],num))
        landmark_ids.extend([landmark_id] * num)

    train_df = pd.DataFrame({'id': image_ids, 'landmark_id': landmark_ids})

train_df["img_folder"] = args.img_path_train
train_df["target"] = 0
# train_df.to_csv(TRAIN_LABEL_FILE, index=False)
train_df

# 非地标图像

In [ ]:
valid_df = pd.read_csv(args.data_path_valid+ args.valid_csv_fn)
valid_df = valid_df[valid_df['landmark_id'] == -1].reset_index(drop=True)
valid_df["img_folder"] = args.img_path_val
valid_df["target"] = -1
valid_df

In [ ]:
test_df = pd.read_csv(args.data_path+ 'sample_submission.csv')
test_df["img_folder"] = args.img_path_test
test_df["target"] = -1
test_df=test_df.rename(columns={'landmarks':'landmark_id'})
test_df['landmark_id']=-1
test_df

In [ ]:
train_df.to_csv('train.csv', index=False)

In [ ]:
# train_df = train_df.iloc[:128,]
# valid_df = valid_df.iloc[:96,]
# test_df = test_df.iloc[:96,]

# 数据集类

In [ ]:

from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch 
import albumentations as A
import multiprocessing as mp 
from tqdm import tqdm
import numpy as np
import cv2

def collate_fn(batch):
    
    input_dict = {}
    target_dict = {}
    
    for key in ['input']:
        input_dict[key] = torch.stack([b[key] for b in batch])
    for key in ['idx']:
        input_dict[key] = torch.stack([b[key] for b in batch]).long()   
    
    for key in ['target']:
        target_dict[key] = torch.stack([b[key] for b in batch]).long()
        
    return input_dict, target_dict

class GLRDataset(Dataset):

    def __init__(self, df, suffix='.jpg', preload=False, aug = None, normalization='simple'):

        self.df = df
        self.aug = aug
        self.normalization = normalization
        self.labels = self.df.target.values
        self.img_folder = self.df.img_folder.values
        self.suffix = suffix
        self.image_names = self.df.id.values
        self.images_cache = {}
        self.images_in_cache = False

        if preload:
            self.preload()
            self.images_in_cache = True
        self.eps = 1e-6

    def __getitem__(self, idx):
        id_ = self.image_names[idx]
        img_folder_ = self.img_folder[idx]
        
        if self.images_in_cache:
            img = self.images_cache[id_]
        else:
            img = self.load_one(id_, img_folder_)
            
        if self.aug:
            img = img.astype(np.uint8)
            img = self.augment(img)
                
        img = img.astype(np.float32)       
        if self.normalization:
            img = self.normalize_img(img)
    
        tensor = self.to_torch_tensor(img)
        
        target = torch.tensor(self.labels[idx])
        feature_dict = {'idx':torch.tensor(idx).long(),
                        'input':tensor,
                       'target':target.float()}
        return feature_dict

    def __len__(self):
        return len(self.image_names)


    def preload(self):
        if self.n_threads > 1:
            with mp.Pool(self.n_threads) as p:
                imgs = p.map(self.load_one,self.id)
            self.images_cache = dict(zip(self.id, imgs))
        else:
            for i in tqdm(self.id):
                self.images_cache[i] = self.load_one(i)

    def load_one(self, id_, img_folder_):
        try:
            img = cv2.imread(img_folder_ + f'{id_[0]}/{id_[1]}/{id_[2]}/{id_}{self.suffix}')
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB )
        except:
            print("FAIL READING IMG", img_folder_ + f'{id_[0]}/{id_[1]}/{id_[2]}/{id_}{self.suffix}')
            img = np.zeros((512,512,3), dtype=np.int8)
        return img

    def augment(self,img):
        img_aug = self.aug(image=img)['image']
        return img_aug.astype(np.float32)

    def normalize_img(self,img):
        
        if self.normalization == 'channel':
            pixel_mean = img.mean((0,1))
            pixel_std = img.std((0,1)) + self.eps
            img = (img - pixel_mean[None,None,:]) / pixel_std[None,None,:]
            img = img.clip(-20,20)

        elif self.normalization == 'channel_mean':
            pixel_mean = img.mean((0,1))
            img = (img - pixel_mean[None,None,:])
            img = img.clip(-20,20)
            
        elif self.normalization == 'image':
            img = (img - img.mean()) / img.std() + self.eps
            img = img.clip(-20,20)
            
        elif self.normalization == 'simple':
            img = img/255
            
        elif self.normalization == 'inception':
            
            mean = np.array([0.5, 0.5 , 0.5], dtype=np.float32)
            std = np.array([0.5, 0.5 , 0.5], dtype=np.float32)
            img = img.astype(np.float32)
            img = img/255.
            img -= mean
            img *= np.reciprocal(std, dtype=np.float32)
            
        elif self.normalization == 'imagenet':
            
            mean = np.array([123.675, 116.28 , 103.53 ], dtype=np.float32)
            std = np.array([58.395   , 57.120, 57.375   ], dtype=np.float32)
            img = img.astype(np.float32)
            img -= mean
            img *= np.reciprocal(std, dtype=np.float32)
            
        else:
            pass
        
        return img
    
    
    def to_torch_tensor(self,img):
        return torch.from_numpy(img.transpose((2, 0, 1)))
    


In [ ]:
import torch
import random
import numpy as np
import os

from typing import Dict, Tuple, Any

from sklearn.metrics import roc_auc_score
from scipy.special import expit, softmax
from sklearn.metrics import precision_score



def set_seed(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def global_average_precision_score(y_true, y_pred, ignore_non_landmarks=False):
    indexes = np.argsort(y_pred[1])[::-1]
    queries_with_target = (y_true < args.n_classes).sum()
    correct_predictions = 0
    total_score = 0.
    i = 1
    # print("y_true:",y_true,"y_pred",y_pred)
    for k in indexes:
        if ignore_non_landmarks and y_true[k] == args.n_classes:
            continue
        if y_pred[0][k] == args.n_classes:
            continue
        relevance_of_prediction_i = 0
        if y_true[k] == y_pred[0][k]:
            correct_predictions += 1
            relevance_of_prediction_i = 1
        precision_at_rank_i = correct_predictions / i
        total_score += precision_at_rank_i * relevance_of_prediction_i
        i += 1
    return 1 / queries_with_target * total_score

def comp_metric(y_true, logits, ignore_non_landmarks=False):
    
    score = global_average_precision_score(y_true, logits, ignore_non_landmarks=ignore_non_landmarks)
    return score

def cos_similarity_matrix(a, b, eps=1e-8):
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
    sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))
    return sim_mt

def get_topk_cossim(test_emb, tr_emb, batchsize = 64, k=10, device='cuda:0',verbose=True):
    tr_emb = torch.tensor(tr_emb, dtype = torch.float32, device=torch.device(device))
    test_emb = torch.tensor(test_emb, dtype = torch.float32, device=torch.device(device))
    vals = []
    inds = []

    for test_batch in test_emb.split(batchsize):
        sim_mat = cos_similarity_matrix(test_batch, tr_emb)
        vals_batch, inds_batch = torch.topk(sim_mat, k=k, dim=1)
        vals += [vals_batch.detach().cpu()]
        inds += [inds_batch.detach().cpu()]
    vals = torch.cat(vals)
    inds = torch.cat(inds)
    return vals, inds
def get_topk_cossim_sub(test_emb, tr_emb, vals_x, batchsize = 64, k=10, device='cuda:0',verbose=True):
    tr_emb = torch.tensor(tr_emb, dtype = torch.float32, device=torch.device(device))
    test_emb = torch.tensor(test_emb, dtype = torch.float32, device=torch.device(device))
    vals_x = torch.tensor(vals_x, dtype = torch.float32, device=torch.device(device))
    vals = []
    inds = []
    for test_batch in tqdm(test_emb.split(batchsize),disable=1-verbose):
        sim_mat = cos_similarity_matrix(test_batch, tr_emb)
        sim_mat = torch.clamp(sim_mat,0,1) - vals_x.repeat(sim_mat.shape[0], 1)
        
        vals_batch, inds_batch = torch.topk(sim_mat, k=k, dim=1)
        vals += [vals_batch.detach().cpu()]
        inds += [inds_batch.detach().cpu()]
    vals = torch.cat(vals)
    inds = torch.cat(inds)
    return vals, inds

In [ ]:
#from pytorchcv.model_provider import get_model as ptcv_get_model
import timm
from torch import nn

import math
import torch
from torch.nn import functional as F
from torch.nn.parameter import Parameter

class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        # stdv = 1. / math.sqrt(self.weight.size(1))
        # self.weight.data.uniform_(-stdv, stdv)

    def forward(self, features):
        cosine = F.linear(F.normalize(features), F.normalize(self.weight))
        return cosine

def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, p_trainable=True):
        super(GeM,self).__init__()
        if p_trainable:
            self.p = Parameter(torch.ones(1)*p)
        else:
            self.p = p
        self.eps = eps

    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)       
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

    
class Backbone(nn.Module):

    
    def __init__(self, name='resnet18', pretrained=True,checkpoint_path=''):
        super(Backbone, self).__init__()

        self.net = timm.create_model(name, pretrained=pretrained,checkpoint_path=checkpoint_path)
        
        if 'regnet' in name:
            self.out_features = self.net.head.fc.in_features
        elif 'csp' in name:
            self.out_features = self.net.head.fc.in_features
        elif 'res' in name: #works also for resnest
            self.out_features = self.net.fc.in_features
        elif 'efficientnet' in name:
            self.out_features = self.net.classifier.in_features
        elif 'densenet' in name:
            self.out_features = self.net.classifier.in_features
        elif 'senet' in name:
            self.out_features = self.net.fc.in_features
        elif 'inception' in name:
            self.out_features = self.net.last_linear.in_features

        else:
            self.out_features = self.net.classifier.in_features

    def forward(self, x):
        x = self.net.forward_features(x)

        return x

    
class Net(nn.Module):
    def __init__(self, args, pretrained=False):
        super(Net, self).__init__()
        
        self.args = args
        self.backbone = Backbone(args.backbone, pretrained=pretrained,checkpoint_path=args.checkpoint_path)
        
        if args.pool == "gem":
            self.global_pool = GeM(p_trainable=args.p_trainable)
        elif args.pool == "identity":
            self.global_pool = torch.nn.Identity()
        else:
            self.global_pool = nn.AdaptiveAvgPool2d(1)

        self.embedding_size = args.embedding_size        
        
        # https://www.groundai.com/project/arcface-additive-angular-margin-loss-for-deep-face-recognition
        if args.neck == "option-D":
            self.neck = nn.Sequential(
                nn.Linear(self.backbone.out_features, self.embedding_size, bias=True),
                nn.BatchNorm1d(self.embedding_size),
                torch.nn.PReLU()
            )
        elif args.neck == "option-F":
            self.neck = nn.Sequential(
                nn.Dropout(0.3),
                nn.Linear(self.backbone.out_features, self.embedding_size, bias=True),
                nn.BatchNorm1d(self.embedding_size),
                torch.nn.PReLU()
            )
        else:
            self.neck = nn.Sequential(
                nn.Linear(self.backbone.out_features, self.embedding_size, bias=False),
                nn.BatchNorm1d(self.embedding_size),
            )
            
        self.head = ArcMarginProduct(self.embedding_size, args.n_classes)
        
        if args.pretrained_weights is not None:
            self.load_state_dict(torch.load(args.pretrained_weights, map_location='cpu'), strict=False)
            print('weights loaded from',args.pretrained_weights)

    def forward(self, input_dict, get_embeddings=False, get_attentions=False):

        x = input_dict['input']
        # print("input",x)
        x = self.backbone(x)
        # print("after backbone",x)
        x = self.global_pool(x)
        x = x[:,:,0,0]
        # print("after pool",x)
        x = self.neck(x)
        # print("embedding",x)
        logits = self.head(x)
        
        if get_embeddings:
            return {'logits': logits, 'embeddings': x}
        else:
            return {'logits': logits}

# 获得embedding

In [ ]:
def get_embeddings(dl, model):
    with torch.no_grad():
        # embeddings = np.zeros((len(dl.dataset) , 512))

        out_list=[]
        total = len(dl)
        # for idx, batch in tqdm(enumerate(dl), total=len(dl)):
        for idx, batch in tqdm(enumerate(dl),total=len(dl)):
            
            # print("idx:",idx)
            input_dict, target_dict = dict_unravel(batch)
            outs = model.forward(input_dict, get_embeddings=True)["embeddings"]
            # embeddings[idx*BATCH_SIZE:idx*BATCH_SIZE+outs.size(0),:] = outs.detach().cpu().numpy()

            out_dict={}
            out_dict['idx']=input_dict['idx']
            out_dict['embeddings']=outs
            out_list.append(out_dict)

    return out_list

In [ ]:
aug = args.test_aug

In [ ]:
tr_ds = GLRDataset(train_df, normalization=args.normalization, aug=aug)
tr_dl = DataLoader(dataset=tr_ds,
                    batch_size=BATCH_SIZE,
                    sampler=SequentialSampler(tr_ds), collate_fn=collate_fn, num_workers=2, pin_memory=True)

In [ ]:
val_ds = GLRDataset(valid_df, normalization=args.normalization, aug=aug)
val_dl = DataLoader(dataset=val_ds,
                    batch_size=BATCH_SIZE,
                    sampler=SequentialSampler(val_ds), collate_fn=collate_fn, num_workers=2, pin_memory=True)

In [ ]:
test_ds = GLRDataset(test_df, normalization=args.normalization, aug=aug)
test_dl = DataLoader(dataset=test_ds,
                    batch_size=BATCH_SIZE,
                    sampler=SequentialSampler(test_ds), collate_fn=collate_fn, num_workers=2, pin_memory=True)

# Inference and submission

In [ ]:
t_m = torch.load(MODEL_PATH)
args.n_classes = t_m['head.weight'].shape[0]
model = Net(args)
model.eval()
model.cuda()
model.load_state_dict(t_m)

In [ ]:
tr_eb = get_embeddings(tr_dl, model)
val_eb = get_embeddings(val_dl, model)
test_eb = get_embeddings(test_dl, model)

In [ ]:
def con_eb(d):
    out_val={}
    for key in d[0].keys():
        out_val[key] = torch.cat([o[key]  for o in d])
    return out_val

In [ ]:
out_tr = con_eb(tr_eb)
out_val = con_eb(val_eb)
out_test =con_eb(test_eb)
for key in out_tr:
    out_tr[key]=out_tr[key].detach().cpu().numpy().astype(np.float32)
for key in out_val:
   out_val[key]=out_val[key].detach().cpu().numpy().astype(np.float32)
for key in out_val:
   out_test[key]=out_test[key].detach().cpu().numpy().astype(np.float32)


In [ ]:
tr_idx = out_tr['idx']
val_idx =out_val['idx']

In [ ]:
tr_names = train_df['landmark_id'][tr_idx].values
val_names = valid_df['landmark_id'][val_idx].values
test_names = test_df['id'].values

In [ ]:
tr_embeddings = out_tr['embeddings']
nonlandmark_embeddings = out_val['embeddings']
test_embeddings = out_test['embeddings']

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer
from sklearn.random_projection import GaussianRandomProjection

f = QuantileTransformer(output_distribution="normal")

f.fit(np.concatenate([test_embeddings],axis=0))


tr_embeddings = f.transform(tr_embeddings)
nonlandmark_embeddings = f.transform(nonlandmark_embeddings)
test_embeddings = f.transform(test_embeddings)

In [ ]:


EMB_SIZE = 512
vals_blend = []
labels_blend = []
inds_blend = []
vals_nl, inds_nl = get_topk_cossim(tr_embeddings, nonlandmark_embeddings, k=5)
vals_nl = vals_nl[:,:].mean(axis=1).detach().cpu().numpy()

vals, inds = get_topk_cossim_sub(test_embeddings, tr_embeddings, vals_nl, k=3)
vals = vals.data.cpu().numpy()
inds = inds.data.cpu().numpy()
# labels = np.concatenate([targets_train[inds[:,i]].reshape(-1,1) for i in range(inds.shape[1])], axis=1)
labels = tr_names[inds]
vals_blend.append(vals)
labels_blend.append(labels)
inds_blend.append(inds)

In [ ]:
from collections import defaultdict

vals_new = []
labels_new = []

for i in tqdm(range(len(vals))):
    cnts = defaultdict(list)

    x = 0
    for j,l in enumerate(labels[i,:]):

        curr = vals[i][j]

        cnts[l].append(curr)

    for k,v in cnts.items():
        cnts[k] = np.sum(v)
        
    labels_new.append(max(cnts, key=cnts.get))
    vals_new.append(cnts[labels_new[-1]])
        
l = np.array(labels_new).reshape(-1)
v = np.array(vals_new).reshape(-1)

In [ ]:
vals_2, inds_2 = get_topk_cossim(test_embeddings, nonlandmark_embeddings, k=11)
# starting from index 1 on val as index 0 is the same image
vals_2 = vals_2[:,1:].mean(axis=1).detach().cpu().numpy()

In [ ]:
import scipy as sp

l3 = pd.Series(l.copy()).reset_index(drop=True)
v3 = v.copy()

v3 -= 1*vals_2

In [ ]:
max_conf = max(v3)
min_conf = min(v3)
submit_confidences = [
    (v - min_conf) / (max_conf - min_conf) for v in v3]

In [ ]:
submit_ids=[]
submit_landmark_ids = []
for i in l3.values:
    submit_landmark_ids.append(i)
for i in test_names:
    submit_ids.append(i)

In [ ]:

submit_landmarks = [
        f'{i} {c:.8f}' for i, c in zip(submit_landmark_ids, submit_confidences)]
        

In [ ]:
submit_df = pd.DataFrame({'id': submit_ids, 'landmarks': submit_landmarks})
submit_df

In [ ]:
submit_df.to_csv('submission.csv', index=False)

In [ ]:
vals_2_s=vals_2.reshape(len(vals_2),1)
vals_2_s=vals_2_s.repeat(vals.shape[1],1)
o_vals = vals -vals_2_s

out_summary={}
out_summary['labels']=labels
out_summary['vals']=o_vals
out_summary['tr_eb']=tr_eb
out_summary['val_eb']=val_eb
out_summary['test_eb']=test_eb

In [ ]:
import pickle
with open('out_summary.p', 'wb') as handle:
    pickle.dump(out_summary, handle)